In [24]:
from Bio.PDB import PDBParser
import numpy as np
import gudhi
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from Bio.PDB.MMCIFParser import MMCIFParser
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.spatial import distance_matrix


def create_df(structure):
    '''creates python data frame of data from .pdb files of 3D protein structures'''
    structure_list = []
    for model in structure: 
        for chain in model:
            for residue in chain:
                for atom in residue:
                    atom_info = {
                        "Model": model.id,
                        "Chain": chain.id,
                        "Residue_Name": residue.get_resname(),
                        "Residue_ID": residue.get_id()[1],  # Sequence number
                        "Atom_Name": atom.get_name(),
                        "Element": atom.element,
                        "X": atom.coord[0],
                        "Y": atom.coord[1],
                        "Z": atom.coord[2]
                    }
                    structure_list.append(atom_info)
    df = pd.DataFrame(structure_list)
    return df

def create_residue_point_cloud(df):
    '''takes in dataframe of .pdb info and creates residue point cloud'''
    # we only want to keep amino acids, no water molecules
    amino_acids = {'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 
               'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 
               'THR', 'TRP', 'TYR', 'VAL'}
    protein_residues = df[df['Residue_Name'].isin(amino_acids)]
    return protein_residues.groupby(["Residue_ID", "Chain"])[['X', 'Y', 'Z']].mean()[['X', 'Y', 'Z']].to_numpy()

def create_3d_plot(point_cloud):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    x = point_cloud[:, 0]
    y = point_cloud[:, 1]
    z = point_cloud[:, 2]

    sc = ax.scatter(x, y, z, c='dodgerblue', s=20, alpha=0.7)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('3D Point Cloud of Hemoglobin Structure')

    plt.tight_layout()
    plt.show()
    return

def make_rips_simplextree_diagram(point_cloud):
    max_edge_length = 25
    rips_complex = gudhi.RipsComplex(points=point_cloud, max_edge_length=max_edge_length)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=3)
    diagram = simplex_tree.persistence()
    return rips_complex, simplex_tree, diagram

def significant_features(diagram, min_persistence = 1):
    '''finds significant features of the plot'''
    significant_features = [(dim, (birth, death)) for dim, (birth, death) in diagram if (death - birth) >= min_persistence]
    return significant_features

def plot(diagram):
    gudhi.plot_persistence_diagram(diagram)
    gudhi.plot_persistence_barcode(diagram)
    return

In [22]:
# deoxygenated hemiglobin data
parser2 = PDBParser(QUIET=True)
deoxy = parser2.get_structure("deoxy", "data/1a3n.pdb") # deoxy
deoxy_df = create_df(deoxy)
deoxy_point_cloud = deoxy_df[['X', 'Y', 'Z']].to_numpy()
deoxy_residue_point_cloud = create_residue_point_cloud(deoxy_df)

# oxygenated hemoglobin data
parser3 = MMCIFParser(QUIET=True)
oxy = parser3.get_structure("oxy1", "data/1hho-assembly1.cif")
oxy_df = create_df(oxy)
oxy1_df = oxy_df[oxy_df["Residue_Name"] != "OXY"] # remove oxygens from point cloud
oxy_point_cloud = oxy_df[['X', 'Y', 'Z']].to_numpy()
oxy_residue_point_cloud = create_residue_point_cloud(oxy1_df)

# sicklecell
parser1 = PDBParser(QUIET=True)
sicklecell = parser1.get_structure("sicklecell", "data/5e83.pdb")
sicklecell_df = create_df(sicklecell)
sicklecell_point_cloud = sicklecell_df[['X', 'Y', 'Z']].to_numpy() # create point cloud (extract 3 dim coordinates)
sicklecell_residue_point_cloud = create_residue_point_cloud(sicklecell_df)

In [ ]:
# create everything
deoxy_rips_complex, deoxy_simplex_tree, deoxy_diagram = make_rips_simplextree_diagram(deoxy_residue_point_cloud)
oxy_rips_complex, oxy_simplex_tree, oxy_diagram = make_rips_simplextree_diagram(oxy_residue_point_cloud)
sicklecell_rips_complex, sicklecell_simplex_tree, sicklecell_diagram = make_rips_simplextree_diagram(sicklecell_residue_point_cloud)

In [26]:
# pull out significant features + plot
deoxy_significant_features = significant_features(deoxy_diagram)
#plot(deoxy_significant_features)
oxy_significant_features = significant_features(oxy_diagram)
#plot(oxy_significant_features)
sicklecell_significant_features = significant_features(sicklecell_diagram)
#plot(sicklecell_significant_features)

In [41]:
def t_test(d1, d2, dim=2):
    for i in range(dim+1):
        dim_values = [d1[i][0] for p in d1 if p[0] == i]
        print(f"For dim={i}:")
    return dim_values
t_test(oxy_diagram, deoxy_diagram)

For dim=0:
For dim=1:
For dim=2:


[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [ ]:
d1 = deoxy_diagram#[0][0]
dim_values = [d1[0][0] for p in d1 if p[0] == 0]
i = 0
deoxy_diagram[i][0] == 

[(2, (12.250899915944101, 14.2210215517213)),
 (2, (8.238189850646675, 9.632872977677337)),
 (2, (7.6455246087346085, 8.915044846712432)),
 (2, (8.170960681882558, 9.420455943475806)),
 (2, (8.702204576725748, 9.940949468779255)),
 (2, (7.590309055834984, 8.771250683341611)),
 (2, (14.148997195641037, 15.298071709392094)),
 (2, (11.25478032881713, 12.352374987192215)),
 (2, (8.849957319425782, 9.931280167734483)),
 (2, (11.33091085338856, 12.289804407585375)),
 (2, (8.511456645037955, 9.465444848871803)),
 (2, (11.562774768569183, 12.432886118297056)),
 (2, (7.63449639263502, 8.490982302446344)),
 (2, (7.636153612786298, 8.463126863850418)),
 (2, (11.34303199103783, 12.16894659608574)),
 (2, (6.832439105377003, 7.618895557944466)),
 (2, (7.602414806577214, 8.384694276220932)),
 (2, (6.865438694093251, 7.644239572491109)),
 (2, (6.7592771197979875, 7.534286806372712)),
 (2, (6.421748673734802, 7.189658148568965)),
 (2, (8.584297772460188, 9.35063968169348)),
 (2, (6.41859880193625, 7.17